In [4]:
import pandas as pd
import math
import numpy as np
import math
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing
from sklearn.preprocessing import normalize # to normalize a matrix
from sklearn.decomposition import PCA
from scipy import linalg
import random

/home/garvit/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
print(max(ratings.overall)) # implies that ratings are on a scale of 13
# train, test = sample_split(ratings)
# print(pd.Series.sort_values(ratings.movie_id).unique())

13


In [1]:
print("users: ",ratings.user_id.drop_duplicates().count()) #number of users
ratings.groupby(['user_id']).count()
print("movies: ",ratings.movie_id.drop_duplicates().count()) #number of movies

NameError: name 'ratings' is not defined

In [7]:
print("minimum movies rated by a user" ,min(ratings.groupby(["user_id"]).overall.count())) # minimum ratings given by any user
print("maximum movies rated by a user" ,max(ratings.groupby(["user_id"]).overall.count())) # maximum ratings given by any user
print("minimum ratings of a movie" ,min(ratings.groupby(["movie_id"]).overall.count())) # minimum ratings of a movie
print("maximum ratings of a movie" ,max(ratings.groupby(["movie_id"]).overall.count())) # maximum ratings of a movie

minimum movies rated by a user 5
maximum movies rated by a user 237
minimum ratings of a movie 1
maximum ratings of a movie 830


In [8]:
# function to crop the dataset such that ratings given by any user>=ru
def min_user_ratings_criteria(ratings, ru):
    # remove all users who have rated <= ru movies
    temp = ratings.groupby(['user_id']).overall.count().reset_index()
#     print(temp)
    users_to_be_removed = temp[temp.overall < ru].user_id.unique()
    print(len(users_to_be_removed))
    new_ratings = ratings[~ratings['user_id'].isin(users_to_be_removed)]
    print(len(new_ratings.user_id.unique()))
    return new_ratings
#     print(len(users_to_be_removed))
# min_ratings_criteria(10,1)

In [9]:
# function to crop the dataset such that every item is rated by atleast ri users
def min_item_ratings_criteria(ratings, ri):
    # remove all items with number of ratings <= ri
    temp = ratings.groupby(['movie_id']).overall.count().reset_index()
#     print(temp)
    items_to_be_removed = temp[temp.overall < ri].movie_id.unique()
    print(len(items_to_be_removed))
    new_ratings = ratings[~ratings['movie_id'].isin(items_to_be_removed)]
    print(len(new_ratings.movie_id.unique()))
    return new_ratings
# min_item_ratings_criteria(5)

In [18]:
# split the datasets into smaller datasets, such that every dataset contains 'size' number of users
def split_dataset(ratings, size):
        df_group = ratings.groupby(['user_id'])
#         print(type(df_group))
        
        df_group_list = list()
        # converting the group of dataframes into list of dataframe for random shuffling
        for index, item in df_group:
            df_group_list.append(item)
        # randomly shuffling the list
        random.shuffle(df_group_list)
        no_of_datasets = int(len(df_group)/size);
#         print(no_of_datasets)
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        datasets_generated = 0 # datasets that have been generated till now
        for item in df_group_list:
            count+=1
            temp = temp.append(item)
            if count==size and datasets_generated < no_of_datasets-1:
                list_datasets.append(temp)
                print("size = ",temp.shape[0])
#                 print(temp.user_id.unique())
                temp = pd.DataFrame()
                datasets_generated += 1
                count=0
        list_datasets.append(temp) # appending the last dataset
        print("size = ",temp.shape[0])
        return list_datasets
    
# testing above function
temp = split_dataset(ratings, 500)
# for item in temp:
#     print(len(item.user_id.unique()))

size =  4962
size =  5490
size =  4791
size =  5083
size =  4930
size =  5331
size =  5020
size =  4949
size =  4878
size =  5621
size =  5186
size =  5915


In [28]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [29]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test

In [112]:
# remove movies with <5 ratings
rating_dash = min_item_ratings_criteria(ratings, 5)
# remove users who have rated <10 items
# rating_dash = min_user_ratings_criteria(rating_dash, 10)
# print(len(rating_dash.movie_id.unique()))
# list conatining all the datasets
list_datasets = split_dataset(rating_dash,1000)
print(rating_dash.shape)

69
907
6
size =  10044
size =  10877
size =  10211
size =  9945
size =  10146
size =  10734
(61957, 8)


In [113]:
# splitting into train and test data
train = list()
test = list()
num_of_datasets = len(list_datasets)
print(num_of_datasets)
for i in range(num_of_datasets):
    print(i)
    t1, t2 = sample_split(list_datasets[i])
    train.append(t1)
    test.append(t2)

6
0
1
2
3
4
5


# Multi-criteria user-user collaborative filtering

In [32]:
# function to find similarity between two users based on Manhattan distance
def manhattan_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += abs(row.criterion1_x - row.criterion1_y) + \
                abs(row.criterion2_x - row.criterion2_y) + \
                abs(row.criterion3_x - row.criterion3_y) + \
                abs(row.criterion4_x - row.criterion4_y) + \
                abs(row.overall_x - row.overall_y)
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# manhattan_similarity(train, 1, 3)

In [33]:
# function to find simialrity between two users based on Euclidean distance
def euclidean_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += math.sqrt((row.criterion1_x - row.criterion1_y)**2 + \
                (row.criterion2_x - row.criterion2_y)**2 + \
                (row.criterion3_x - row.criterion3_y)**2 + \
                (row.criterion4_x - row.criterion4_y)**2) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
# euclidean_similarity(train, 1, 3)

In [34]:
# function to find simialrity between two users based on Chebyshev distance
def chebyshev_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += max(abs(row.criterion1_x - row.criterion1_y), \
                abs(row.criterion2_x - row.criterion2_y), \
                abs(row.criterion3_x - row.criterion3_y), \
                abs(row.criterion4_x - row.criterion4_y), \
                abs(row.overall_x - row.overall_y))
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# chebyshev_similarity(train, 1, 3)

In [51]:
# function to predict the rating given by user to item
# neighbours = -1 implies that consider similarity with all possible users
def predict(df, user, item, similarity, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        sim = similarity(df, user, v) # find appropriate similarity measure between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating
        

In [119]:
# print(test[0].head(26))

In [18]:
# print(train)
print(predict(train[0], 1, 879, euclidean_similarity))
print(predict(train[0], 1, 879, manhattan_similarity))
print(predict(train[0], 1, 879, chebyshev_similarity))

10.37336757999115
9.959976730657361
9.569398389388915


In [35]:
# function to calculate the mean average error between two vectors
def mean_average_error(list1, list2):
    sum1=0
    for a,b in zip(list1,list2):
        if(b==0):            # to handle the unpredicted ratings
            continue
        sum1 += abs(a-b)
    error = sum1/len(list1)
    return error
mean_average_error((1,2),(10,20))

13.5

In [36]:
# function to replace entries in a list with zero/one based on some threshold
def replace_with_zero_one(list1, threshold):
    length = len(list1)
    rlist = list()
    i = 0
    while i < length:
        if(list1[i]==0):   # to handle the unpredicted ratings
            list1[i]=-1
            continue
        if(list1[i]>threshold):
            rlist.append(1)
        else:
            rlist.append(0)
        i += 1
    return rlist

In [37]:
# function to calculate the precision between two lists
# list2 is the predicted list
def precision(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fp = 0 # false positives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==1 and a==0):
            fp+=1
    precision = tp/(tp + fp)
    return precision

# precision([20,5,14], [1,16,20], 13)

In [38]:
# function to calculate the recall between two lists
# list2 is the predicted list
def recall(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fn = 0 # false negatives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==0 and a==1):
            fn += 1
#     print("tp = ",tp)
#     print("fn = ",fn)
    recall = tp/(tp + fn)
    return recall

# recall([20,15,14], [1,16,15], 13)

In [39]:
# function to calculate F1-score between two lists
def f1_score(list1, list2, threshold):
    p = precision(list1, list2, threshold)
    r = recall(list1, list2, threshold)
    f1 = (2*p*r)/(p+r)
    return f1

In [108]:
# predicting on our training dataset
predicted = list() # for storing the predicted ratings
# appending 7 empty lists
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1
unpredicted_count = list()
# print(predicted1)

In [ ]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    cnt = 0
    for index, row in test[i].iterrows():
        predicted_rating = predict(train[i], row.user_id, row.movie_id,chebyshev_similarity)
#         print(predicted_rating)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
        print(predicted_rating)
    print("predicted %d",i)
    unpredicted_count.append(cnt)

In [177]:
# predicting on our training dataset1
print("test-size = ", test[1].shape[0])
predicted[1] = list() # for storing the predicted ratings
i=0
for index, row in test[1].iterrows():
    predicted_rating = predict(train[1], row.user_id, row.movie_id,euclidean_similarity, 20)
    print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[1].append(predicted_rating)
print(predicted[1])

test-size =  3717
0
Continuing
1
2
3
4
5
6
7
8
9
Continuing
10
Continuing
11
Continuing
12
13
14
15
16
Continuing
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
We cannot predict
36
Continuing
37
38
Continuing
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
Continuing
62
63
64
65
66
67
68
Continuing
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
Continuing
87
88
Continuing
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
Continuing
143
144
145
146
147
We cannot predict
148
Continuing
We cannot predict
149
Continuing
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
Continuing
177
178
Continuing
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
Continuing
197
198
199
200
201
202
203
204
205
206
207
208
Continuing
209


1514
1515
Continuing
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
Continuing
1526
Continuing
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
Continuing
1555
1556
Continuing
1557
Continuing
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
Continuing
1571
Continuing
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
Continuing
1582
1583
1584
1585
1586
Continuing
1587
1588
1589
1590
1591
We cannot predict
1592
Continuing
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
Continuing
1619
Continuing
1620
Continuing
1621
1622
1623
Continuing
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
Continuing
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
Continuing
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672


2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
Continuing
2867
2868
2869
2870
2871
2872
Continuing
2873
Continuing
2874
2875
2876
2877
2878
2879
2880
Continuing
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
Continuing
2893
Continuing
2894
2895
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
Continuing
2917
2918
2919
2920
Continuing
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
Continuing
2934
2935
2936
Continuing
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946
Continuing
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
2974
2975
2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015

In [178]:
# print("unpredicted = ",unpredicted_count[1])
print(len(predicted[1]))
threshold = 7
ttest = test[1].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[1], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[1]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

3403
mae =  3.061022683151661
precision =  0.7712137486573577
recall =  0.8284615384615385
f1-score =  0.7988132764694974


# Singular Value Decomposition (SVD)

In [16]:
# function to convert data into mxn matrix m -> number of users, n -> number of movies
# the matrix is normalized to take care of missing ratings
# after normalizations missing ratings can be safely assumed to be zero
# df dataframe should contain columns as 'user_id', 'movie_id', 'rate'
def convert_to_normalized_matrix(df, rating_column):
    no_of_movies = len(ratings.movie_id.unique())
    no_of_users = len(ratings.user_id.unique())
    matrix = np.zeros((no_of_users, no_of_movies))
    for index, row in df.iterrows():
        matrix[row.user_id-1][row.movie_id-1] = row[rating_column]
#     print("completed")
    return matrix,normalize_matrix(matrix)

# function to normalize a matrix by subtracting the average rating of the user
def normalize_matrix(matrix):
#     matrix[matrix == 0] = np.nan
#     matrix = np.ma.array(matrix, mask=np.isnan(matrix))

    # normalize the matrix
    normalized_matrix = np.zeros((len(matrix), len(matrix[0])))
    i=0
#     print(matrix)
    num_users = len(matrix)
    while i < num_users:
        baseline = np.sum(matrix[i])
        baseline /= np.count_nonzero(matrix[i])
        # subtract baseline value from all non-zero ratings
        normalized_matrix[i][:] = [x-baseline if x>0 else x for x in matrix[i]]
        i+=1
    return normalized_matrix
    
    
# g,h = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', 'overall']])
# print(h)

In [17]:
# function to apply SVD on normalized user-item matrix
def svd(ratings, rating_column):
    original,X = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', rating_column]], rating_column)
    print("matrix created, now decomposing it")
    u,s,v = np.linalg.svd(X) # Singular Value Decomposition
    return original,u,s,v

In [19]:
# function to crop the decomposed matrices to top-k latent features
def crop_matrices(u, s, v, k):
    # calculate average rating of the user according to 'matrix'
    cropped_u = u[:,:k] # crop matrix u
    cropped_s = s[:k] # crop array s
    cropped_v = v[:k,:] # crop matrix v
    return cropped_u, cropped_s, cropped_v
# crop_matrices(u,s,v,15)

In [46]:
# to predict the rating given by 'user' to 'item' based on SVD
# u, s, v are the decomposed matrices
# matrix is the parent matrix that is decomposed
def predict_svd(matrix, u, s, v, user, item):
    # calculate average rating of 'user' according to 'matrix'
    baseline = np.sum(matrix[user-1])
    baseline /= np.count_nonzero(matrix[user-1])
#     print("average -> ", baseline)
    p = u[user-1] # latent-features values of the user
    # s is the weight of latent-features
    q = np.transpose(v)[item-1]
    
    return baseline+sum(p*s*q)
    
# matrix,u,s,v = svd(ratings,'overall')
# print("completed")
# a,b,c = crop_matrices(u,s,v,15)
# print("completed")
# predict_svd(matrix,a,b,c,1,1)

In [61]:
# creating the decomposed matrices
matrix,u,s,v = svd(train[0], 'overall')

/home/garvit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it


In [103]:
# croppping the matrices to appropriate no. of latent-features
a,b,c = crop_matrices(u,s,v,15)

In [105]:
# testing the svd implementation
# fingers crossed
# predicting on our training dataset1
print("test-size = ", test[0].shape[0])
predicted[0] = list() # for storing the predicted ratings
i=0
for index, row in test[0].iterrows():
    predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
#     print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[0].append(predicted_rating)
# print(predicted[0])

test-size =  1698


In [114]:
# creating empty lists for storing the predicted values
predicted = list() # for storing the predicted ratings
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1

In [115]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    matrix,u,s,v = svd(train[i], 'overall')
    a,b,c = crop_matrices(u,s,v,15)
    for index, row in test[i].iterrows():
        predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
#         print(predicted_rating)
    print("predicted ",i)
#     unpredicted_count.append(cnt)

/home/garvit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it
predicted  0
matrix created, now decomposing it
predicted  1
matrix created, now decomposing it
predicted  2
matrix created, now decomposing it
predicted  3
matrix created, now decomposing it
predicted  4
matrix created, now decomposing it
predicted  5


In [125]:
# Evaluating the model
print(len(predicted[5]))
threshold = 7
ttest = test[5].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[5], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[5]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

3725
mae =  2.581395961901881
precision =  0.7973529411764706
recall =  0.943612948137835
f1-score =  0.8643392316276104
